In [ ]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# ==========================
# 1️⃣ Load and label dataset
# ==========================
df_binary = pd.read_csv("cleaned_dataset.csv")
df_binary["cleaned_review_text"] = df_binary["cleaned_review_text"].astype(str).fillna("")
print("Class distribution (binary):")
print(df_binary["sentiment"].value_counts())

# ==========================
# 2️⃣ Tokenize and encode
# ==========================
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df_binary["cleaned_review_text"])


# Vocabulary size
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # +1 because indexing starts at 1
print(f"Vocabulary size: {vocab_size}")

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df_binary["cleaned_review_text"])

# ==========================
# Step 2: Maximum sequence length
# ==========================
sequence_lengths = [len(seq) for seq in sequences]
max_len = max(sequence_lengths)
print(f"\nMaximum sequence length: {max_len}")



X = pad_sequences(sequences, maxlen=max_len, padding='post')
print("\nPadded sequences (first 5 samples):")
for i in range(5):
    print(f"Sample {i+1}: {X[i]}")
print(f"\nShape of padded input: {X.shape}")


encoder = LabelEncoder()
y = encoder.fit_transform(df_binary["sentiment"])
y_cat = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y_cat
)

# ==========================
# 3️⃣ Compute class weights
# ==========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

# ==========================
# 4️⃣ Build BiLSTM model
# ==========================


model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])
model.build(input_shape=(None, max_len))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# ==========================
# 5️⃣ Train with early stopping
# ==========================
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=64,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {acc:.2f}")

# ==========================
# 6️⃣ Save model & tokenizer
# ==========================
model.save("sentiment_model.keras")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)


Class distribution (binary):
sentiment
Positive    4448
Negative     406
Name: count, dtype: int64
Vocabulary size: 3614

Maximum sequence length: 102

Padded sequences (first 5 samples):
Sample 1: [ 102   45    4   13    2   71    7  234  143  634 1216   15 1217  635
   21  301 1218  636  277  278  197   26  637    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
Sample 2: [ 20   2  20   7 302   4 120  12  92  15   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0  

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 102, 128)       │       462,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ (None, 256)            │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 742,338 (2.83 MB)

 Trainable params: 742,338 (2.83 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.7519 - loss: 0.6745 - val_accuracy: 0.8790 - val_loss: 0.3933
Epoch 2/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.8283 - loss: 0.4805 - val_accuracy: 0.8919 - val_loss: 0.2524
Epoch 3/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 57s 700ms/step - accuracy: 0.9097 - loss: 0.2613 - val_accuracy: 0.9189 - val_loss: 0.2248
Epoch 4/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 35s 592ms/step - accuracy: 0.9634 - loss: 0.1626 - val_accuracy: 0.8571 - val_loss: 0.3469
Epoch 5/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 29s 584ms/step - accuracy: 0.9490 - loss: 0.1498 - val_accuracy: 0.9138 - val_loss: 0.2907
Epoch 6/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 29s 600ms/step - accuracy: 0.9786 - loss: 0.0960 - val_accuracy: 0.9073 - val_loss: 0.3267
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - accuracy: 0.9195 - loss: 0.2250

✅ Test Accuracy: 0.92


In [ ]:
def analyze_brand_sentiment(brand_name):
    # Load components
    model = load_model("sentiment_model.keras")
    with open("tokenizer.pkl", "rb") as f:
        tokenizer = pickle.load(f)
    with open("label_encoder.pkl", "rb") as f:
        encoder = pickle.load(f)

    # Filter reviews from the balanced dataset
    brand_reviews = df_binary[df_binary["brand"].str.lower() == brand_name.lower()]["cleaned_review_text"]
    if brand_reviews.empty:
        return f"No reviews found for brand '{brand_name}'."

    # Prepare sequences
    sequences = tokenizer.texts_to_sequences(brand_reviews)
    padded = pad_sequences(sequences, maxlen=max_len, padding='post')

    # Predict
    preds = model.predict(padded, verbose=0)
    pred_labels = encoder.inverse_transform(np.argmax(preds, axis=1))

    # Count results
    counts = pd.Series(pred_labels).value_counts().to_dict()
    result = {sent: counts.get(sent, 0) for sent in ["Positive", "Negative"]}
    return result


In [ ]:
print(analyze_brand_sentiment("Redmi"))


{'Positive': 49, 'Negative': 10}


In [ ]:
result = analyze_brand_sentiment("Redmi")

# Compute percentages
total = sum(result.values())
percentages = {k: round((v / total) * 100, 2) for k, v in result.items()}

print("📊 Sentiment Percentages for Redmi:")
for sentiment, pct in percentages.items():
    print(f"{sentiment}: {pct}%")

📊 Sentiment Percentages for Redmi:
Positive: 83.05%
Negative: 16.95%


In [ ]:
# ============================================================
# 💬 Continue after the sentiment analysis percentages section
# ============================================================
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from huggingface_hub import login
import re

# --- Login to Hugging Face ---
login("huggingface token")

# --- Load model and tokenizer ---
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer_hf = AutoTokenizer.from_pretrained(model_name)
model_hf = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# --- Helper to build emotion prompt ---
def build_emotion_prompt(brand_name, brand_reviews):
    joined_reviews = "\n".join(brand_reviews)
    prompt = f"""
You are an expert emotion classifier. Analyze the following reviews for brand "{brand_name}".
Each review expresses emotions such as: enjoyment, satisfaction, anger, disappointment, confusion, excitement, trust, or surprise.

Identify the **most prominent overall emotion** across all reviews.

⚠️ Respond ONLY with one word — the emotion (e.g., enjoyment, anger, trust, confusion, disappointment, satisfaction, excitement, surprise).

Reviews:
{joined_reviews}

Now output only one word — the most prominent emotion:
"""
    return prompt.strip()


# --- Combined function for both Sentiment + Emotion ---
def full_brand_analysis(brand_name):
    # --- Sentiment from BiLSTM ---
    sentiment_result = analyze_brand_sentiment(brand_name)
    total = sum(sentiment_result.values())
    percentages = {k: round((v / total) * 100, 2) for k, v in sentiment_result.items()}

    # --- Average rating ---
    brand_reviews_df = df_binary[df_binary["brand"].str.lower() == brand_name.lower()]
    if brand_reviews_df.empty:
        return f"No reviews found for brand '{brand_name}'."

    avg_rating = brand_reviews_df["rating"].mean()

    # --- Emotion from Mistral ---
    brand_reviews = brand_reviews_df["review_text"].tolist()
    emotion_prompt = build_emotion_prompt(brand_name, brand_reviews)
    inputs = tokenizer_hf(emotion_prompt, return_tensors="pt", truncation=True).to(model_hf.device)

    with torch.no_grad():
        emotion_output = model_hf.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer_hf.eos_token_id)

    emotion_text = tokenizer_hf.decode(emotion_output[0], skip_special_tokens=True)
    emotion_match = re.search(
        r"\b(enjoyment|satisfaction|anger|disappointment|confusion|excitement|trust|surprise)\b",
        emotion_text,
        re.IGNORECASE
    )
    emotion = emotion_match.group(1).capitalize() if emotion_match else "Unknown"

    # --- Print nicely formatted report ---
    print(f"\n⭐ Average Rating for {brand_name}: {avg_rating:.2f}/5\n")
    print(f"📊 Sentiment Report for {brand_name}:")
    for sentiment, pct in percentages.items():
        print(f"  {sentiment}: {pct}%")

    print(f"\n💫 Most Prominent Emotion: {emotion}")

    return {
        "average_rating": round(avg_rating, 2),
        "sentiment_percentages": percentages,
        "dominant_emotion": emotion
    }


# --- Example run ---
final_result = full_brand_analysis("Redmi")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



⭐ Average Rating for Redmi: 4.12/5

📊 Sentiment Report for Redmi:
  Positive: 83.05%
  Negative: 16.95%

💫 Most Prominent Emotion: Enjoyment
